## [2022 Day 16 - Proboscidea Volcanium](https://adventofcode.com/2022/day/16)

An exhaustive brute force search of the solution space here is intractable. There are 30 * 56 * 2^56 potential states to search. But since there is no point in opening valves with non-zero flow, we can use this intuition to prune the solution space into something less computationally intensive, because only 15 valves have flow. 

In [2]:
import re
import heapq

Now, we need to parse the input into a dictionary of `room: (flow_rate, [neighbors])`

In [27]:
rooms = {}

with open('data/day16.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]

    for line in lines:
        valves = re.findall(r'[A-Z]{2}', line)
        room, neighbors = valves[0], valves[1:]

        rate = re.findall(r'\d+', line)

        flow = int(rate[0])
        if flow > 0 or room == 'AA':
            rooms[room] = flow

# example 
rooms

{'JI': 21,
 'DM': 3,
 'XS': 18,
 'WT': 23,
 'KI': 9,
 'II': 19,
 'VC': 24,
 'DU': 12,
 'ZK': 10,
 'XF': 25,
 'LC': 4,
 'IY': 22,
 'TE': 11,
 'VF': 13,
 'BD': 17,
 'AA': 0}

Next, we need to find the shortest difference between each pair of nodes in the graph. 

In [10]:
def shortest_path(source, target):
	seen = set()
	queue = [(0, source)]
	while queue:
		d, p = heapq.heappop(queue)
		if p == target:
			return d
		seen.add(p)
		for q in rooms[p][1]:
			if q not in seen:
				heapq.heappush(queue, (d + 1, q))

distances = {(a, b): shortest_path(a, b) for a in rooms for b in rooms}
distances[('JI', 'DM')]

4

So now we have a map of `room` to `flow_rate` and the shortest path from any one node in the graph to any other node in the graph in the dictionary `distances`. 

Now we can do a depth first search to explore all possible routes and find the max

In [29]:
def dfs(distances, rooms, current_valve, opened, time_remaining) -> int:
	# base case: we've run out of time and need to return the flow
	if time_remaining < 0:
		return 0 
	else:
		# We want to open this valve since we filtered out all of the
		# valves with flow of 0 above
		opened_1 = opened | {current_valve}
		
		# remove the opened valves from the set of candidate rooms because we do
		# not want to revisit it
		valves_1 = set(rooms) - opened_1 
		
		# this is how much this valve will contribute for the remainder of the time
		flow_1 = rooms[current_valve] * time_remaining
		
		flow_2 = max((dfs(distances, rooms, v, opened_1, time_remaining - distances[current_valve, v] - 1) for v in valves_1), default=0)
		
		return flow_1 + flow_2	
	
	
	
	


dfs(distances, rooms, 'AA', set(), 30)

2059

In [12]:
opened = {'A', 'B', 'C'}

In [14]:
opened | {'A'}

{'A', 'B', 'C'}

In [18]:
opened | {'D'}

{'A', 'B', 'C', 'D'}